In [1]:
import pandas as pd
import numpy as np
import math
import os
import pickle
from collections import OrderedDict
from IPython.display import display, clear_output, Markdown, HTML

from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.io as pio

from simulator import *
from simulator_plotting import *

init_notebook_mode(connected=True)

In [2]:
pairs = [('CTX', 'SAM'), ('AM', 'AMC'), ('ZOX', 'CXM')]
pairs = [[dataset2.loc[item] for item in pair] for pair in pairs]

In [3]:
# naive basin size of a single genotype within a fitness landscape
def basin_size(landscape, genotype, num=10, **kwargs):
    size = 0
    if isinstance(landscape, list):
        name = ' + '.join([ls.name for ls in landscape])
        landscape = [ls.tolist() for ls in landscape]
    else:
        name = landscape.name
        landscape = landscape.tolist()
    param = dict(kwargs)
    for seed in range(16):
        param['seed'] = seed
        for i in range(num):
            results = simulate(landscape, **param)
            if results['actual_path'][-1] == genotype:
                size += 1
    return size / num

In [4]:
# get basin sizes for each genotype in a fitness landscape
def basin_sizes(landscape, k=9, **kwargs):
    sizes = []
    for i in range(16):
        gen = format(i, '04b')
        bs = basin_size(landscape, gen, carrying_cap=10**k, prob_mutation=10**(-(k-1)), **kwargs)
        #print('The basin size of {} is {}'.format(gen, bs))
        print('.', end='')
        sizes.append(bs)
    print()
    return sizes

In [10]:
# generate and serialize some collections of basin sizes
with open('basin_sizes.pickle', 'a+b') as f:
    f.seek(0)
    try:
        data = {}
        data.update(pickle.load(f))
    except:
        data = {}
        
# basin sizes for all drugs in the 2nd dataset
if 'dataset2' not in data:
    data['dataset2'] = {}
for name, ls in dataset2.iterrows():
    if name not in data['dataset2']:
        data['dataset2'][name] = basin_sizes(ls)

# basin sizes for all drugs in the 2nd dataset at k=10^6
if 'dataset2_6' not in data:
    data['dataset2_6'] = {}
for name, ls in dataset2.iterrows():
    if name not in data['dataset2_6']:
        data['dataset2_6'][name] = basin_sizes(ls, k=6)
        
# basin sizes for the pairs at f=200
if 'pairs' not in data:
    data['pairs'] = {}
for i, ls in enumerate(pairs):
    if i not in data['pairs']:
        data['pairs'][i] = basin_sizes(ls, k=9, frequency=200)

# basin sizes for the pairs at f=200 / k=10^6
if 'pairs_6' not in data:
    data['pairs_6'] = {}
for i, ls in enumerate(pairs):
    if i not in data['pairs_6']:
        data['pairs_6'][i] = basin_sizes(ls, k=6, frequency=200)
        
with open('basin_sizes.pickle', 'wb') as f:
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

bs = data

# Basin size

For a given genotype, the basin size is the number of other genotypes it can be reached from.

Reachability--One genotype is reachable from another if a simulation run on a starting population consisting entirely of the first genotype results in fixation of the second genotype after 1200 timesteps. (We could define this in other ways; for example, the abundance of the second genotype reaching a certain threshold at some point during the simulation could mean it is reachable.)

Since simulations can have different outcomes, the code above essentially takes an average of the basin size for each genotype after running 10 simulations.

## Basin size for static landscapes

Non-zero basin sizes appear to correspond to the local and global optima that we [found earlier](https://nbviewer.jupyter.org/url/jops.bol.ucla.edu/18-11-01/Oct-31-many-sims-w-switching.ipynb#K=10^9).

In [55]:
data=[]
for name in dataset2.index:
    data.append(go.Scatter(
        y=bs['dataset2'][name],
        name=name
    ))
fig = tools.make_subplots(rows=len(data), cols=1, print_grid=False)
for i, d in enumerate(data, 1):
    fig.append_trace(d, i, 1)
    fig['layout'][f'xaxis{i}'].update(
        ticktext=[format(i, '04b') for i in range(16)],
        tickvals=list(range(16)),
        range=[-0.1,15.1],
        zeroline=False
    )
    fig['layout'][f'yaxis{i}'].update(
        range=[-1,17],
        tickvals=[0,8,16],
        title=d.name,
        showticklabels=False
    )
    
fig['layout'].update(
    height=1200,
    showlegend=False,
    title='Basin sizes at K=10<sup>9</sup>'
)
iplot(fig, show_link=False)

In [58]:
data = [
    go.Scatter(y=bs['dataset2'][dataset2.index[0]], name='10^9'),
    go.Scatter(y=bs['dataset2_6'][dataset2.index[0]], name='10^6')
]
buttons = []
for name in dataset2.index:
    buttons.append(dict(
        args=[{'y': [bs['dataset2'][name], bs['dataset2_6'][name]]}],
        label=name,
        method='restyle'
    ))

updatemenus=list([
    dict(
        buttons=buttons,
        direction = 'down',
        pad = {'r': 10, 't': 10},
        showactive = True,
        x = 0.1,
        xanchor = 'left',
        y = 1.1,
        yanchor = 'top' 
    ),
])

layout = go.Layout(
    title='Comparison of basin sizes at two carrying capacities',
    xaxis=dict(
        ticktext=[format(i, '04b') for i in range(16)],
        tickvals=list(range(16)),
        title='Genotype',
        zeroline=False
    ),
    yaxis=dict(
        title='Basin size',
        range=[-1,17]
    ),
    updatemenus=updatemenus
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, show_link=False)

## Basin size for dynamic landscapes

In [81]:
layout = go.Layout(
    xaxis=dict(
        ticktext=[format(i, '04b') for i in range(16)],
        tickvals=list(range(16)),
        title='Genotype',
        zeroline=False
    ),
    yaxis=dict(
        title='Basin size',
        range=[-1,17]
    )
)

for i, pair in enumerate(pairs):
    name = ' + '.join([ls.name for ls in pair])
    data = [
        go.Scatter(y=bs['pairs'][i], name='K=10^9'),
        go.Scatter(y=bs['pairs_6'][i], name='K=10^6', line=dict(dash='dash'))
    ]
    layout.update(title=name)
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, show_link=False)